In [81]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from scipy.spatial.distance import euclidean

from mode_imputer import KNNImputerMode

In [82]:
data = pd.read_csv('./Financial Well-Being Survey Data/NFWBS_PUF_2016_data.csv')

print(f'The dataset contains {data.shape[0]} rows and {data.shape[1]} features.')

The dataset contains 6394 rows and 217 features.


# Data cleaning

In [83]:
data.set_index('PUF_ID', inplace=True)

## Dropping irrelevant features

We can remove features that were used to feature engineering

Engineered features: 
- FWBScore: created from FWB questions
- FSscore: created from FS questions
- LMscore: created from FINKKNOWL questions + FKcorrect
- KHscore: created from KHKNOWL questions + KHcorrect
- ON1correct: OBJNUMERACY1
- ON2correct: OBJNUMERACY2

In [84]:
FWB_cols = list(filter(lambda x: 'FWB' in x, data.columns.values))
FWB_drop = FWB_cols[1:]
data.drop(FWB_drop, axis = 1, inplace = True)

list(filter(lambda x: 'FWB' in x, data.columns.values))

['FWBscore']

In [85]:
FS_cols = list(filter(lambda x: 'FS' in x, data.columns.values))
FS_cols = FS_cols[1:]
data.drop(FS_cols, axis = 1, inplace = True)

list(filter(lambda x: 'FS' in x, data.columns.values))

['FSscore']

In [86]:
LM_cols = list(filter(lambda x: 'FINKNOWL' in x, data.columns.values))

data.drop(LM_cols, axis = 1, inplace = True)

list(filter(lambda x: 'FINKNOWL' in x, data.columns.values))

[]

In [87]:
FKcorrect_cols = ['FK1correct', 'FK2correct', 'FK3correct']

data.drop(FKcorrect_cols, axis = 1, inplace = True)

In [88]:
KH_cols = list(filter(lambda x: 'KHKNOWL' in x, data.columns.values))

data.drop(KH_cols, axis = 1, inplace = True)

list(filter(lambda x: 'KHKNOWL' in x, data.columns.values))

[]

In [89]:
KHcorrect_cols = ['KH1correct', 'KH2correct', 'KH3correct', 'KH4correct', 'KH5correct', 'KH6correct', 'KH7correct', 'KH8correct', 'KH9correct']

data.drop(KHcorrect_cols, axis = 1, inplace = True)

In [90]:
OBJNUMcols = list(filter(lambda x: 'OBJNUMERACY' in x, data.columns.values))

data.drop(OBJNUMcols, axis = 1, inplace = True)

list(filter(lambda x: 'OBJNUMERACY' in x, data.columns.values))

[]

In [91]:
data.head()

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
10350,2,3,5,5,6,55,44,5,4,3,...,1,4,8,0,0,0,0,1,0,0.367292
7740,1,3,6,6,6,51,43,5,4,3,...,1,2,3,0,0,0,0,2,0,1.327561
13699,1,3,4,3,4,49,42,5,3,3,...,1,4,9,0,0,0,1,2,1,0.835156
7267,1,3,6,6,6,49,42,-1,-1,-1,...,1,3,7,0,0,0,0,1,0,1.410871
7375,1,3,4,4,4,49,42,4,3,3,...,1,2,4,0,0,1,0,4,1,4.260668


## Missing values

__-5: County not known__


Use PPREG9, PPINCIMP and PPEDUC to impute PCTLT200FPL
PPEDUC: ordinal
<br>
PPINCIMP: ordinal
<br>
PPREG9: categorical

distance to find nearest neighbors: euclidean for PPINC, PPEDUC  + one hot for PPREG9

In [92]:
# replace by missing values
data = data.replace(-5, np.nan)

In [93]:
nrows = data[data['PCTLT200FPL'].isna()].shape[0]
print(f'nr of rows with missing value: {nrows}')

nr of rows with missing value: 395


In [94]:
def distance(X, Y, missing_values = np.nan):
    
    X[np.isnan(X)] = 0
    Y[np.isnan(Y)] = 0

    eucl_dist = euclidean(X[[0, 1]], Y[[0, 1]])

    return eucl_dist + (X[2] != Y[2])

In [95]:
imputer = KNNImputerMode(n_neighbors = 5, metric = distance)

cols = ['PPEDUC', 'PPINCIMP', 'PPREG9', 'PCTLT200FPL']

new_data = imputer.fit_transform(data[cols])

new_data = pd.DataFrame(new_data, columns = cols, index = data.index)

In [96]:
new_data['PCTLT200FPL'].value_counts()

0.0    4842
1.0    1552
Name: PCTLT200FPL, dtype: int64

In [97]:
data.loc[data['PCTLT200FPL'].isna(), 'PCTLT200FPL'] = new_data.loc[data['PCTLT200FPL'].isna(), 'PCTLT200FPL'].copy()

__-4: Response not written to the database due to error__: SWB only (because FWB dropped) ->DROP

In [98]:
data[(data == -4).any(axis = 1)]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
12173,1,3,-4,-4,-4,-4,48,7,4,3,...,1,3,5,0,0,0,0,1,0.0,1.253964


In [99]:
data.drop(12173, axis = 0, inplace = True)

__-3: Invalid response/ Incoherent data__

- Drop the only row with KIDS_2 = -3
- Replace -3 with -2 for SOCSEC2 because -3 are caused by invalid age, therefore -2 is the adequate label.
- Replace -3 with the mode for SOCSEC3

KIDS_2 drop

In [100]:
data[data['KIDS_2'] == -3]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
13237,1,2,3,7,7,52,50,5,4,4,...,1,2,3,0,0,1,1,3,0.0,1.690435


In [101]:
data.drop(13237, axis = 0, inplace = True)

SOCSEC2

In [102]:
data['SOCSEC2'] = data['SOCSEC2'].replace(-3, -2).copy()

SOCSEC3

In [103]:
data.loc[data['SOCSEC3'].isna(), 'SOCSEC3'] = data['SOCSEC3'].mode()[0]

__8: I can't recall__

HSLOC: Where respondent attended high school
- 1: US and territories
- 2: Outside the US
- 8: Can't recall

What do to?
- impute with mode

In [104]:
data.loc[data['HSLOC'] == 8, 'HSLOC'] = data['HSLOC'].mode()[0]

In [105]:
data['HSLOC'].value_counts()

 1    5406
-1     644
 2     342
Name: HSLOC, dtype: int64

__-1: Not answered/ Left blank__
<br>
__98: Don't know__
<br>
__99: Prefer not to say__

First, let's replace the missing values with NaN

In [106]:
data = data.replace(-1, np.nan)
data = data.replace(98, np.nan)
data = data.replace(99, np.nan)

In [107]:
data[data.isna().sum(axis = 1) >= 5]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
7267,1,3,6.0,6.0,6.0,49.0,42.0,NaN,NaN,NaN,...,1,3,7,0,0,0,0,1,0.0,1.410871
8303,1,3,7.0,7.0,7.0,43.0,58.0,4.0,3.0,3.0,...,1,3,7,0,0,0,1,3,0.0,2.497838
9182,1,3,6.0,NaN,NaN,50.0,43.0,4.0,4.0,3.0,...,1,4,9,0,0,0,0,2,0.0,0.647301
11082,1,3,6.0,6.0,5.0,62.0,55.0,4.0,5.0,4.0,...,1,3,5,0,0,0,0,3,0.0,0.455829
8309,2,3,3.0,4.0,2.0,49.0,37.0,NaN,4.0,3.0,...,1,3,5,0,0,0,0,1,0.0,0.597089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11078,3,1,3.0,4.0,5.0,26.0,44.0,4.0,3.0,3.0,...,1,2,3,0,1,0,0,1,0.0,1.379959
12353,1,3,5.0,7.0,3.0,62.0,26.0,4.0,3.0,3.0,...,1,3,5,0,0,0,0,2,0.0,1.745759
12297,3,3,4.0,5.0,7.0,55.0,45.0,5.0,4.0,3.0,...,1,2,4,0,0,0,1,2,0.0,0.883213


Now let's drop the rows with more than 5 missing values

In [108]:
data.drop(data[data.isna().sum(axis = 1) >= 5].index, axis = 0, inplace = True)

In [109]:
data.shape

(6025, 171)

Now, let's see which columns have missing values

In [110]:
data.isna().sum(axis = 0).index

Index(['sample', 'fpl', 'SWB_1', 'SWB_2', 'SWB_3', 'FWBscore', 'FSscore',
       'SUBKNOWL1', 'ACT1_1', 'ACT1_2',
       ...
       'PPMSACAT', 'PPREG4', 'PPREG9', 'PPT01', 'PPT25', 'PPT612', 'PPT1317',
       'PPT18OV', 'PCTLT200FPL', 'finalwt'],
      dtype='object', length=171)

And how many missing values per column

In [111]:
# how many missing values per column that has missing values

data.isna().sum(axis = 0)[data.isna().sum(axis = 0) > 0]

SWB_1              12
SWB_2              37
SWB_3              38
SUBKNOWL1          34
ACT1_1             14
                   ..
KIDS_4              7
EMPLOY             44
RETIRE              2
MILITARY           15
Military_Status    31
Length: 79, dtype: int64

Get all columns with less than 100 missing values

In [112]:
# get all columns with less than 100 missing values

data.isna().sum(axis = 0)[(data.isna().sum(axis = 0) > 0) & (data.isna().sum(axis = 0) < 100)].index

Index(['SWB_1', 'SWB_2', 'SWB_3', 'SUBKNOWL1', 'ACT1_1', 'ACT1_2', 'FINGOALS',
       'SAVEHABIT', 'FRUGALITY', 'AUTOMATED_1', 'AUTOMATED_2', 'ASK1_1',
       'ASK1_2', 'SUBNUMERACY2', 'SUBNUMERACY1', 'CHANGEABLE', 'GOALCONF',
       'ENDSMEET', 'HOUSING', 'LIVINGARRANGEMENT', 'CONSPROTECT1',
       'CONSPROTECT2', 'CONSPROTECT3', 'EARNERS', 'VOLATILITY', 'SNAP',
       'MATHARDSHIP_6', 'COLLECT', 'REJECTED_1', 'REJECTED_2', 'ABSORBSHOCK',
       'BENEFITS_3', 'BENEFITS_4', 'BENEFITS_5', 'FRAUD2', 'COVERCOSTS',
       'MANAGE2', 'PAIDHELP', 'PAREDUC', 'FINSOC2_3', 'FINSOC2_5', 'FINSOC2_6',
       'MATERIALISM_1', 'MATERIALISM_2', 'MATERIALISM_3', 'HEALTH',
       'SCFHORIZON', 'DISCOUNT', 'MEMLOSS', 'DISTRESS', 'SELFCONTROL_1',
       'SELFCONTROL_2', 'SELFCONTROL_3', 'OUTLOOK_1', 'OUTLOOK_2', 'PEM',
       'HOUSESAT', 'SOCSEC1', 'SOCSEC2', 'SOCSEC3', 'HHEDUC', 'KIDS_1',
       'KIDS_2', 'KIDS_3', 'KIDS_4', 'EMPLOY', 'RETIRE', 'MILITARY',
       'Military_Status'],
      dtype='object'

In [113]:
data.isna().sum(axis = 0)[data.isna().sum(axis = 0) > 100].index

Index(['HOUSERANGES', 'VALUERANGES', 'MORTGAGE', 'SAVINGSRANGES', 'BORROW_1',
       'BORROW_2', 'HSLOC', 'CONNECT', 'LIFEEXPECT', 'KIDS_NoChildren'],
      dtype='object')

__Central tendency imputations__

SWB_1, SWB_2, SUBKNOWL1, ACT1_2, ASK1_1, SUBNUMERACY1, LIVINGARRANGEMENT, CONSPROTECT1, ABSORBSHOCK, PAREDUC, MATERIALISM_3, HEALTH, SELFCONTROL_1, SELFCONTROL_2, SELFCONTROL_3, OUTLOOK_1, OUTLOOK_2, PEM: skewed -> median

SWB_3, ACT1_1, SAVEHABIT, FRUGALITY, ENDSMEET, CONSPROTECT2, MATHARDSHIP_6, HOUSESAT: staricase -> mode

AUTOMATED_1, AUTOMATED_2, ASK1_2, SUBNUMERACY2, CHANGEABLE, GOALCONF, EARNERS, MATERIALISM,_1, MATERIALISM_2, SCFHORIZON, DISTRESS, HHEDUC, EMPLOY, RETIRE (cuidado com -2): gaussian distribution -> mode

Categorical:

FINGOALS, AUTOMATED_2, HOUSING, CONSPROTECT3, VOLATILITY, SNAP, COLLECT, REJECTED_1, REJECTED_2, BENEFITS_3, BENEFITS_4, BENEFITS_5, FRAUD2, COVERCOSTS, MANAGE2, PAIDHELP (cuidado com -2), FINSOC2_3, FINSOC2_5, FINSOC2_6, DISCOUNT, MEMLOSS, SOCSEC1, SOCSEC2, SOCSEC3 (cuidado com -3 e -2), KIDS_1, KIDS_2, KIDS_3, KIDS_4, EMPLOY, MILITARY, Military_Status: mode

In [114]:
data['PAIDHELP'].value_counts()

-2.0    3113
 0.0    2709
 1.0     201
Name: PAIDHELP, dtype: int64

In [115]:
skewed_ordinal = ['SWB_1', 'SWB_2', 'SUBKNOWL1', 'ACT1_2', 'ASK1_1', 'SUBNUMERACY1', 'LIVINGARRANGEMENT', 'CONSPROTECT1', 'ABSORBSHOCK', 'PAREDUC', 'MATERIALISM_3', 'HEALTH', 'SELFCONTROL_1', 'SELFCONTROL_2', 'SELFCONTROL_3', 'OUTLOOK_1', 'OUTLOOK_2', 'PEM']

for feat in skewed_ordinal:
    data.loc[data[feat].isna(), feat] = data[feat].median()

staircase_ordinal = ['SWB_3', 'ACT1_1', 'SAVEHABIT', 'FRUGALITY', 'ENDSMEET', 'CONSPROTECT2', 'MATHARDSHIP_6', 'HOUSESAT']

for feat in staircase_ordinal:
    data.loc[data[feat].isna(), feat] = data[feat].mode()[0]



# excluding RETIRE because it has -2 values
gaussian_ordinal = ['AUTOMATED_1', 'AUTOMATED_2', 'ASK1_2', 'SUBNUMERACY2', 'CHANGEABLE', 'GOALCONF', 'EARNERS', 'MATERIALISM_1', 'MATERIALISM_2', 'SCFHORIZON', 'DISTRESS', 'HHEDUC', 'EMPLOY']

for feat in gaussian_ordinal:
    data.loc[data[feat].isna(), feat] = data[feat].mode()[0]

valid_data = data[data['RETIRE'] != -2].copy()
data.loc[data['RETIRE'].isna(), 'RETIRE'] = valid_data['RETIRE'].mode()[0]



# excluding PAIDHELP, SOCSEC1, SOCSEC2 and SOCSEC3 because they have -2 values
categorical = ['FINGOALS', 'AUTOMATED_2', 'HOUSING', 'CONSPROTECT3', 'VOLATILITY', 'SNAP', 'COLLECT', 'REJECTED_1', 'REJECTED_2', 'BENEFITS_3', 'BENEFITS_4', 'BENEFITS_5', 'FRAUD2', 'COVERCOSTS', 'MANAGE2', 'FINSOC2_3', 'FINSOC2_5', 'FINSOC2_6', 'DISCOUNT', 'MEMLOSS', 'KIDS_1', 'KIDS_2', 'KIDS_3', 'KIDS_4', 'EMPLOY', 'MILITARY', 'Military_Status']

for feat in categorical:
    data.loc[data[feat].isna(), feat] = data[feat].mode()[0]

valid_data = data[data['PAIDHELP'] != -2].copy()
data.loc[data['PAIDHELP'].isna(), 'PAIDHELP'] = valid_data['PAIDHELP'].mode()[0]

valid_data = data[data['SOCSEC1'] != -2].copy()
data.loc[data['SOCSEC1'].isna(), 'SOCSEC1'] = valid_data['SOCSEC1'].mode()[0]

valid_data = data[data['SOCSEC2'] != -2].copy()
data.loc[data['SOCSEC2'].isna(), 'SOCSEC2'] = valid_data['SOCSEC2'].mode()[0]

valid_data = data[data['SOCSEC3'] != -2].copy()
data.loc[data['SOCSEC3'].isna(), 'SOCSEC3'] = valid_data['SOCSEC3'].mode()[0]


__Other imputation approaches__

['HOUSERANGES', 'VALUERANGES', 'MORTGAGE', 'SAVINGSRANGES', 'BORROW_1',
       'BORROW_2', 'HSLOC', 'CONNECT', 'LIFEEXPECT', 'KIDS_NoChildren']

__HOUSERANGES__

'About how much do you pay for your home each month?'

In [154]:
data[data['HOUSERANGES'].isna()]

,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FSscore,SUBKNOWL1,ACT1_1,ACT1_2,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
PUF_ID,,,,,,,,,,,,,,,,,,,,,
11079,1,3,5.0,4.0,7.0,51.0,54.0,5.0,5.0,3.0,...,1,2,4,0,0,1,1,3,0.0,1.370522
7852,1,2,5.0,6.0,7.0,50.0,49.0,5.0,4.0,4.0,...,1,3,7,0,0,0,1,3,1.0,1.292576
11221,1,1,3.0,5.0,4.0,37.0,32.0,4.0,5.0,5.0,...,1,4,9,0,0,1,0,3,0.0,1.025192
12990,2,2,7.0,7.0,7.0,67.0,79.0,7.0,5.0,5.0,...,1,3,5,0,0,0,0,1,0.0,0.866999
10549,1,3,6.0,5.0,5.0,62.0,48.0,5.0,4.0,3.0,...,1,2,3,0,0,0,0,2,0.0,0.616246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10348,1,3,5.0,7.0,6.0,56.0,42.0,5.0,4.0,3.0,...,0,2,4,0,0,0,0,2,0.0,0.328092
12971,3,3,4.0,5.0,2.0,52.0,43.0,5.0,4.0,2.0,...,1,4,9,0,0,0,0,1,0.0,0.545312
12095,3,2,5.0,5.0,6.0,67.0,50.0,4.0,5.0,5.0,...,1,2,3,1,1,0,1,2,0.0,0.690902
